TRY
top_k_categorical_accuracy
https://stackoverflow.com/questions/47887533/keras-convolution-along-samples
https://keras.io/layers/wrappers/#timedistributed

In [1]:
input_width = 160
input_height = 100
channels = 5
class_number = 12
data_path = "D:\\Python\\Wormax_learn2\\preprocessed_data_local_notshuffled\\"
model_name = 'models/worm_single_adam_5ch_1'
look_forward = 1
import keras

Q:\Program Files\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
from keras.models import Sequential, Model
from keras import layers, models, optimizers
import keras.backend as K

def actual_acc(y_true, y_pred):
    return K.equal(K.argmax(y_pred), K.argmax(y_true))

def define_model():
    model = models.Sequential()
        
    model.add(layers.Conv2D(32, (3, 3), activation='relu',
                            input_shape=(input_height, input_width, channels)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.Dense(class_number, activation='softmax'))    
           
    model.compile(optimizer=optimizers.Adam(),
                  loss='categorical_crossentropy',
                  metrics=[actual_acc])
    
    model.summary()
    return model

In [3]:
model = define_model()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 98, 158, 32)       1472      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 49, 79, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 47, 77, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 23, 38, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 21, 36, 128)       73856     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 10, 18, 128)       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 8, 16, 128)        147584    
__________

In [4]:
# little prepocessing
from math import atan2, pi
from image_preproc import prepare_image

def get_angle(x, y):
    return atan2(y, x)

def get_direction(x, y, n_classes = 12):
    return round(get_angle(x, y)/2/pi*n_classes)%n_classes

In [5]:
import numpy as np
import os
import random
from functools import reduce
from keras.utils import to_categorical

# Training and validation
data_ratio = 0.7

def generator(data_dir, num_classes, role, shuffle=True, batch_size=128):
    
    listdir = []
    listdir = filter(lambda x: os.path.isfile, os.listdir(data_dir))
    listdir = np.array(list(listdir))
    random.shuffle(listdir)
    
    #print('Found {} files for {}'.format(len(listdir), role))
    
    file_i = 0
    while 1:
        data = np.load(data_dir + listdir[file_i])
        file_i = (file_i+1) if file_i+1<len(listdir) else 0
        
        if role == 'train':
            data = data[:int(round(len(data)*data_ratio))]
        elif role == 'validation':
            data = data[int(round(len(data)*data_ratio)):]
        else:
            raise 'bad role parameter'
        
        # Generating y
        data_targets = np.zeros((len(data)-look_forward, 12))
        for i in range(len(data_targets)):
            data_targets[i] = np.array(to_categorical(get_direction(*data[i+look_forward][1][:2]), num_classes=num_classes))
        data = data[:-1]
        
        # Only X
        data_features = data[:,0]
        for i in range(len(data_features)):
            data_features[i] = np.dstack((data_features[i],
                                          prepare_image(data_features[i])))
        
        indexes = np.arange(len(data_features)-1)
        
        if shuffle:
            np.random.shuffle(indexes)
        
        for i in range(0, len(data)-look_forward-batch_size, batch_size):
            samples = data_features.take(indexes[i:i+batch_size],axis=0)
            targets = data_targets.take(indexes[i:i+batch_size],axis=0)
            
            # Weird reshape cuz bug # actually not bug, initial data array is not tensor(idk how to fix)
            samples2 = np.zeros((batch_size, input_height, input_width, channels))
            for j, sample in enumerate(samples):
                samples2[j] = sample
            
            # will not work without this
            samples2 = samples2 / 255
            yield samples2, np.array(targets)

In [6]:
# count class instances count for balancing
if False:
    i = 0
    classes = np.zeros((class_number))
    for samples, targets in generator(data_path, class_number, 'train', batch_size=1024):
        for j in targets:
            classes += j
        i += 1
        if i == 1000:
            break
    classes

In [7]:
import numpy as np

train_generator = generator(data_path, class_number, 'train', batch_size=16)
validation_generator = generator(data_path, class_number, 'validation', batch_size=32)

print(next(train_generator)[0].shape)
print(next(train_generator)[1].shape)

(6999,)
(16, 100, 160, 5)
(16, 12)


### tensorboard --logdir=D:\Python\Keras\Wormax\log_dir

In [8]:
class_weight = {0: 1.0,
                 1: 1.62,
                 2: 2.68,
                 3: 3.36,
                 4: 2.51,
                 5: 1.53,
                 6: 1.0,
                 7: 1.37,
                 8: 2.16,
                 9: 2.61,
                 10: 2.04,
                 11: 1.3}

callbacks = [
    keras.callbacks.TensorBoard(
        log_dir='log_dir\\' + model_name
    ),
    keras.callbacks.ModelCheckpoint(
        filepath=model_name + '.h5',
        monitor='val_loss',
        save_best_only=True,
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss', 
        factor=0.1,                              
        patience=50, 
        min_lr=0.00001
    )
]

steps_per_epoch = 100
history = model.fit_generator(train_generator,
                            steps_per_epoch=steps_per_epoch,
                            epochs=500,
                            validation_data=validation_generator,
                            validation_steps=int(round(steps_per_epoch/data_ratio*(1-data_ratio))),
                            shuffle=True,
                            class_weight=class_weight,
                            callbacks=callbacks
                        )

Epoch 1/500
100/100 [==============================] - 14s 141ms/step - loss: 4.2615 - actual_acc: 0.0825 - val_loss: 2.5036 - val_actual_acc: 0.0581
Epoch 2/500
100/100 [==============================] - 15s 149ms/step - loss: 3.8664 - actual_acc: 0.2338 - val_loss: 2.4006 - val_actual_acc: 0.1860
Epoch 3/500
100/100 [==============================] - 4s 43ms/step - loss: 3.3390 - actual_acc: 0.3625 - val_loss: 2.4515 - val_actual_acc: 0.1642
Epoch 4/500
100/100 [==============================] - 15s 154ms/step - loss: 2.9676 - actual_acc: 0.4494 - val_loss: 2.6484 - val_actual_acc: 0.1693
Epoch 5/500
100/100 [==============================] - 22s 221ms/step - loss: 3.2889 - actual_acc: 0.3419 - val_loss: 2.2608 - val_actual_acc: 0.2413
Epoch 6/500
100/100 [==============================] - 17s 173ms/step - loss: 3.2385 - actual_acc: 0.3550 - val_loss: 2.3686 - val_actual_acc: 0.2137
Epoch 7/500
100/100 [==============================] - 4s 43ms/step - loss: 2.8617 - actual_acc: 0.435

100/100 [==============================] - 4s 44ms/step - loss: 2.7398 - actual_acc: 0.4719 - val_loss: 2.1019 - val_actual_acc: 0.2682
Epoch 38/500
100/100 [==============================] - 99s 988ms/step - loss: 2.5127 - actual_acc: 0.5187 - val_loss: 2.1443 - val_actual_acc: 0.2747
Epoch 39/500
100/100 [==============================] - 4s 44ms/step - loss: 2.3369 - actual_acc: 0.5394 - val_loss: 2.2466 - val_actual_acc: 0.3670
Epoch 40/500
100/100 [==============================] - 201s 2s/step - loss: 2.8480 - actual_acc: 0.4031 - val_loss: 2.1679 - val_actual_acc: 0.3837
Epoch 41/500
100/100 [==============================] - 4s 43ms/step - loss: 2.6932 - actual_acc: 0.4306 - val_loss: 2.0794 - val_actual_acc: 0.2791
Epoch 42/500
100/100 [==============================] - 95s 949ms/step - loss: 2.4302 - actual_acc: 0.5025 - val_loss: 2.1043 - val_actual_acc: 0.2747
Epoch 43/500
100/100 [==============================] - 4s 43ms/step - loss: 2.1478 - actual_acc: 0.5506 - val_loss

100/100 [==============================] - 96s 956ms/step - loss: 1.5865 - actual_acc: 0.6631 - val_loss: 1.7797 - val_actual_acc: 0.3983
Epoch 75/500
100/100 [==============================] - 100s 1s/step - loss: 2.5311 - actual_acc: 0.4856 - val_loss: 1.5904 - val_actual_acc: 0.4571
Epoch 76/500
100/100 [==============================] - 93s 926ms/step - loss: 2.3878 - actual_acc: 0.5206 - val_loss: 2.1932 - val_actual_acc: 0.2754
Epoch 77/500
100/100 [==============================] - 91s 912ms/step - loss: 2.1409 - actual_acc: 0.5775 - val_loss: 2.3458 - val_actual_acc: 0.2464
Epoch 78/500
100/100 [==============================] - 4s 43ms/step - loss: 1.8990 - actual_acc: 0.6181 - val_loss: 1.9473 - val_actual_acc: 0.3554
Epoch 79/500
100/100 [==============================] - 115s 1s/step - loss: 2.2570 - actual_acc: 0.5544 - val_loss: 1.8764 - val_actual_acc: 0.3641
Epoch 80/500
100/100 [==============================] - 4s 43ms/step - loss: 2.5837 - actual_acc: 0.4731 - val_lo

100/100 [==============================] - 131s 1s/step - loss: 2.8063 - actual_acc: 0.4169 - val_loss: 1.7167 - val_actual_acc: 0.3612
Epoch 111/500
100/100 [==============================] - 4s 43ms/step - loss: 2.6658 - actual_acc: 0.4581 - val_loss: 1.7182 - val_actual_acc: 0.3794
Epoch 112/500
100/100 [==============================] - 63s 626ms/step - loss: 2.5086 - actual_acc: 0.4850 - val_loss: 1.8584 - val_actual_acc: 0.3576
Epoch 113/500
100/100 [==============================] - 44s 440ms/step - loss: 2.4009 - actual_acc: 0.5019 - val_loss: 1.8481 - val_actual_acc: 0.3983
Epoch 114/500
100/100 [==============================] - 59s 593ms/step - loss: 2.4713 - actual_acc: 0.4862 - val_loss: 1.7889 - val_actual_acc: 0.3735
Epoch 115/500
100/100 [==============================] - 52s 517ms/step - loss: 2.4592 - actual_acc: 0.5112 - val_loss: 1.7224 - val_actual_acc: 0.4157
Epoch 116/500
100/100 [==============================] - 4s 43ms/step - loss: 2.2175 - actual_acc: 0.5563 

Epoch 147/500
100/100 [==============================] - 4s 43ms/step - loss: 2.7788 - actual_acc: 0.4612 - val_loss: 1.8773 - val_actual_acc: 0.3416
Epoch 148/500
100/100 [==============================] - 15s 145ms/step - loss: 2.7288 - actual_acc: 0.4531 - val_loss: 1.8142 - val_actual_acc: 0.3808
Epoch 149/500
100/100 [==============================] - 42s 419ms/step - loss: 2.5549 - actual_acc: 0.4694 - val_loss: 1.7846 - val_actual_acc: 0.3808
Epoch 150/500
100/100 [==============================] - 4s 43ms/step - loss: 2.3600 - actual_acc: 0.5162 - val_loss: 1.8161 - val_actual_acc: 0.3735
Epoch 151/500
100/100 [==============================] - 21s 206ms/step - loss: 2.0716 - actual_acc: 0.5737 - val_loss: 1.6622 - val_actual_acc: 0.4004
Epoch 152/500
100/100 [==============================] - 21s 212ms/step - loss: 1.9412 - actual_acc: 0.5906 - val_loss: 1.6188 - val_actual_acc: 0.4491
Epoch 153/500
100/100 [==============================] - 29s 292ms/step - loss: 1.9319 - act

...

Epoch 327/500
100/100 [==============================] - 31s 310ms/step - loss: 2.7355 - actual_acc: 0.4513 - val_loss: 1.6702 - val_actual_acc: 0.4222
Epoch 328/500
100/100 [==============================] - 22s 219ms/step - loss: 3.0204 - actual_acc: 0.3844 - val_loss: 1.8505 - val_actual_acc: 0.3714
Epoch 329/500
100/100 [==============================] - 4s 43ms/step - loss: 2.8953 - actual_acc: 0.3950 - val_loss: 1.9254 - val_actual_acc: 0.3510
Epoch 330/500
100/100 [==============================] - 24s 239ms/step - loss: 2.8598 - actual_acc: 0.4125 - val_loss: 1.6834 - val_actual_acc: 0.4113
Epoch 331/500
100/100 [==============================] - 49s 485ms/step - loss: 2.6995 - actual_acc: 0.4438 - val_loss: 1.6003 - val_actual_acc: 0.4419
Epoch 332/500
100/100 [==============================] - 4s 43ms/step - loss: 2.6045 - actual_acc: 0.4562 - val_loss: 1.3640 - val_actual_acc: 0.5705
Epoch 333/500
100/100 [==============================] - 23s 226ms/step - loss: 2.5813 - act

Epoch 363/500
100/100 [==============================] - 4s 43ms/step - loss: 2.7964 - actual_acc: 0.4300 - val_loss: 1.6403 - val_actual_acc: 0.4288
Epoch 364/500
100/100 [==============================] - 22s 223ms/step - loss: 2.8262 - actual_acc: 0.4150 - val_loss: 1.7131 - val_actual_acc: 0.3968
Epoch 365/500
100/100 [==============================] - 22s 219ms/step - loss: 2.7995 - actual_acc: 0.4019 - val_loss: 1.6845 - val_actual_acc: 0.4128
Epoch 366/500
100/100 [==============================] - 32s 323ms/step - loss: 2.5194 - actual_acc: 0.4838 - val_loss: 1.4309 - val_actual_acc: 0.4629
Epoch 367/500
100/100 [==============================] - 23s 226ms/step - loss: 1.9716 - actual_acc: 0.5913 - val_loss: 1.4916 - val_actual_acc: 0.4411
Epoch 368/500
100/100 [==============================] - 4s 43ms/step - loss: 1.9550 - actual_acc: 0.6044 - val_loss: 1.4787 - val_actual_acc: 0.4724
Epoch 369/500
100/100 [==============================] - 23s 226ms/step - loss: 1.9081 - act

Epoch 400/500
100/100 [==============================] - 49s 489ms/step - loss: 2.5933 - actual_acc: 0.4637 - val_loss: 1.3907 - val_actual_acc: 0.5400
Epoch 401/500
100/100 [==============================] - 22s 224ms/step - loss: 2.3770 - actual_acc: 0.5031 - val_loss: 1.3904 - val_actual_acc: 0.5422
Epoch 402/500
100/100 [==============================] - 4s 43ms/step - loss: 2.3940 - actual_acc: 0.5181 - val_loss: 1.2603 - val_actual_acc: 0.5589
Epoch 403/500
100/100 [==============================] - 23s 232ms/step - loss: 2.3705 - actual_acc: 0.5294 - val_loss: 1.3185 - val_actual_acc: 0.5596
Epoch 404/500
100/100 [==============================] - 30s 297ms/step - loss: 2.5423 - actual_acc: 0.4919 - val_loss: 1.7036 - val_actual_acc: 0.3888
Epoch 405/500
100/100 [==============================] - 15s 149ms/step - loss: 2.7624 - actual_acc: 0.4206 - val_loss: 1.6991 - val_actual_acc: 0.4121
Epoch 406/500
100/100 [==============================] - 21s 209ms/step - loss: 2.7663 - a

100/100 [==============================] - 48s 481ms/step - loss: 2.6179 - actual_acc: 0.4769 - val_loss: 1.7076 - val_actual_acc: 0.4012
Epoch 436/500
100/100 [==============================] - 4s 43ms/step - loss: 2.5888 - actual_acc: 0.4631 - val_loss: 1.5661 - val_actual_acc: 0.4281
Epoch 437/500
100/100 [==============================] - 22s 223ms/step - loss: 2.6397 - actual_acc: 0.4744 - val_loss: 1.5422 - val_actual_acc: 0.4462
Epoch 438/500
100/100 [==============================] - 4s 43ms/step - loss: 2.6510 - actual_acc: 0.4656 - val_loss: 1.6224 - val_actual_acc: 0.4186
Epoch 439/500
100/100 [==============================] - 49s 494ms/step - loss: 2.5486 - actual_acc: 0.4738 - val_loss: 1.5518 - val_actual_acc: 0.4629
Epoch 440/500
100/100 [==============================] - 4s 43ms/step - loss: 2.4410 - actual_acc: 0.4819 - val_loss: 1.5030 - val_actual_acc: 0.4949
Epoch 441/500
100/100 [==============================] - 23s 226ms/step - loss: 2.3014 - actual_acc: 0.5200 

100/100 [==============================] - 22s 223ms/step - loss: 2.3603 - actual_acc: 0.5106 - val_loss: 1.6712 - val_actual_acc: 0.3910
Epoch 472/500
100/100 [==============================] - 4s 43ms/step - loss: 2.5118 - actual_acc: 0.4612 - val_loss: 1.7477 - val_actual_acc: 0.3721
Epoch 473/500
100/100 [==============================] - 22s 223ms/step - loss: 2.4544 - actual_acc: 0.4937 - val_loss: 1.7418 - val_actual_acc: 0.3779
Epoch 474/500
100/100 [==============================] - 30s 303ms/step - loss: 2.0426 - actual_acc: 0.5831 - val_loss: 1.7617 - val_actual_acc: 0.4230
Epoch 475/500
100/100 [==============================] - 23s 226ms/step - loss: 1.6366 - actual_acc: 0.6587 - val_loss: 1.7190 - val_actual_acc: 0.4222
Epoch 476/500
100/100 [==============================] - 4s 43ms/step - loss: 1.5588 - actual_acc: 0.6769 - val_loss: 1.7030 - val_actual_acc: 0.4288
Epoch 477/500
100/100 [==============================] - 24s 237ms/step - loss: 1.5101 - actual_acc: 0.691

In [9]:
from keras import backend as K
K.clear_session()